# **Visual Transformer 2D**


In [1]:
import os
import sys
import re
from glob import glob
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
from torchmetrics.classification import MulticlassMatthewsCorrCoef
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.nn.functional import kl_div
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
import torchvision
from torchvision.datasets.mnist import MNIST
from torchvision.transforms import ToTensor

import open3d as o3
import math
import yaml
import argparse


np.random.seed(0)
torch.manual_seed(0)


sys.path.append('../')
from src.models.VisualTransformerEncoder import *
from src.models.VisualTransformerDecoder import *
from src.models.MultiHeadAttentionBlock import *
from src.models.VisualTransformerGenerator import *
from src.utils import features, utils
from src.data.dataset import DataMNIST

from tqdm import tqdm, trange

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# TEMP for supressing pytorch user warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
print('MPS is build: {}'.format(torch.backends.mps.is_built()))
print('MPS Availability: {}'.format(torch.backends.mps.is_available()))
DEVICE = 'cuda' if torch.cuda.is_available() else 'mps'
print('Device is set to :{}'.format(DEVICE))
torch.seed = 42

MPS is build: True
MPS Availability: True
Device is set to :mps


# DATA LOADING

In [4]:
with open('../config/config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [5]:
data = DataMNIST(**config["data_parameters"])
data.setup()

In [6]:
train_dataloader = data.train_dataloader()
val_dataloader = data.val_dataloader()

## PRE & POST PROCESSING SCRIPTS

In [ ]:
#def patchify(images, n_patches):
#    n, c, h, w = images.shape
#
#    assert h == w, "Patchify method is implemented for square images only"
#
#    patches = torch.zeros(n, n_patches**2, h * w * c // n_patches**2)
#    patch_size = h // n_patches
#
#    for idx, image in enumerate(images):
#        for i in range(n_patches):
#            for j in range(n_patches):
#                patch = image[
#                    :,
#                    i * patch_size : (i + 1) * patch_size,
#                    j * patch_size : (j + 1) * patch_size,
#                ]
#                patches[idx, i * n_patches + j] = patch.flatten()
#    return patches
#
#def depatchify(patches, n_patches, chw):
#    # size of patches is expected to be n, h, w
#    patch_h = chw[1] // n_patches
#    patch_w = chw[2] // n_patches
#    n = patches.shape[0]
#
#    images_recovered = torch.zeros(n, chw[0], chw[1], chw[2],  device='mps')
#
#    for idx, patch in enumerate(patches):
#        
#        image_r = torch.empty(0, chw[2],  device='mps')
#        
#        for i in range(n_patches):
#            #patch_r_i_1 = patch[i, :].view(patch_h, patch_w)
#            row_tensor = torch.empty(patch_h, 0,  device='mps')
#            for j in range(n_patches):
#                patch_r_row = patch[i*n_patches+j, :].view(patch_h, patch_w)
#                row_tensor = torch.cat((row_tensor, patch_r_row), dim=1)
#            
#            image_r = torch.cat((image_r, row_tensor), dim=0)
#
#        images_recovered[idx] = image_r
#    return images_recovered
#
#def get_positional_embeddings(sequence_length, d):
#    result = torch.ones(sequence_length, d)
#    for i in range(sequence_length):
#        for j in range(d):
#            result[i][j] = (
#                np.sin(i / (10000 ** (j / d)))
#                if j % 2 == 0
#                else np.cos(i / (10000 ** ((j - 1) / d)))
#            )
#    return result

def TensorToImageGrid(images_batch, rows, cols):
    grid = torchvision.utils.make_grid(images_batch, nrow=cols)
    grid = grid.permute(1, 2, 0).cpu().numpy()
    plt.figure(figsize=(cols, rows))
    plt.imshow(grid, cmap='gray')
    plt.axis('off')
    return plt.show()

def TensorToImage(image):
    plt.imshow(image.numpy(), cmap='gray')
    return plt.show()

In [ ]:
#class SelfAttentionLayer(nn.Module):
#    '''
#    hidden_d: number of hidden dimensions
#    n_heads: number of attention heads
#    
#    '''
#    def __init__(self, hidden_d, n_heads=2):
#        super(SelfAttentionLayer, self).__init__()
#        self.hidden_d = hidden_d
#        self.n_heads = n_heads
#
#        assert hidden_d % n_heads == 0, f"{hidden_d} cannot be divided into {n_heads} heads!"
#
#        d_head = int(hidden_d / n_heads) ## Hidded dimensions per head
#        
#        # Define the linear transformation layers for Query, Key, and Value
#        self.q_mappings = nn.ModuleList(
#            [nn.Linear(d_head, d_head) for _ in range(self.n_heads)]
#        )
#        self.k_mappings = nn.ModuleList(
#            [nn.Linear(d_head, d_head) for _ in range(self.n_heads)]
#        )
#        self.v_mappings = nn.ModuleList(
#            [nn.Linear(d_head, d_head) for _ in range(self.n_heads)]
#        )
#        self.d_head = d_head
#        self.softmax = nn.Softmax(dim=-1)
#
#    def forward(self, sequences):
#        
#        # Sequences has shape (N, seq_length, token_dim)
#        # We go into shape    (N, seq_length, n_heads, token_dim / n_heads)
#        # And come back to    (N, seq_length, item_dim)  (through concatenation)
#
#        result = []
#        for sequence in sequences:
#            seq_result = []
#            for head in range(self.n_heads):
#                q_mapping = self.q_mappings[head]
#                k_mapping = self.k_mappings[head]
#                v_mapping = self.v_mappings[head]
#
#                seq = sequence[:, head * self.d_head : (head + 1) * self.d_head]
#                q, k, v = q_mapping(seq), k_mapping(seq), v_mapping(seq)
#
#                attention = self.softmax(q @ k.T / (self.d_head**0.5)) # Apply the softmax function to calculated the scaled dot-product attention scores to get attention weights
#                # attention = F.dropout(attention, p=0.1) # Drop out to attention - optional
#                seq_result.append(attention @ v)  # Calculate the weighted sum using the attention weights
#            result.append(torch.hstack(seq_result))
#        return torch.cat([torch.unsqueeze(r, dim=0) for r in result])
#
#

In [ ]:
#class MultiHeadAttention(nn.Module):
#    def __init__(self, hidden_d, n_heads):
#        super(MultiHeadAttention, self).__init__()
#        assert hidden_d % n_heads == 0, f"{hidden_d} cannot be divided into {n_heads} heads!"
#
#        self.hidden_d = hidden_d
#        self.n_heads = n_heads
#        self.d_k = hidden_d // n_heads ## Hidded dimensions per head
#        
#        # Define the linear transformation layers for Query, Key, and Value
#        self.W_q = nn.Linear(hidden_d, hidden_d)
#        self.W_k = nn.Linear(hidden_d, hidden_d)
#        self.W_v = nn.Linear(hidden_d, hidden_d)
#        self.W_o = nn.Linear(hidden_d, hidden_d)
#        
#    def scaled_dot_product_attention(self, Q, K, V, mask=None):
#        # Apply the softmax function to calculated the scaled dot-product attention scores to get attention weights
#        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
#        if mask is not None:
#            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
#        attn_probs = torch.softmax(attn_scores, dim=-1)
#        output = torch.matmul(attn_probs, V) # Calculate the weighted sum using the attention weights
#        return output
#        
#    def split_heads(self, x):
#        batch_size, seq_length, hidden_d = x.size()
#        return x.view(batch_size, seq_length, self.n_heads, self.d_k).transpose(1, 2)
#        
#    def combine_heads(self, x):
#        batch_size, _, seq_length, d_k = x.size()
#        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.hidden_d)
#        
#    def forward(self, Q, K, V, mask=None):
#        Q = self.split_heads(self.W_q(Q))
#        K = self.split_heads(self.W_k(K))
#        V = self.split_heads(self.W_v(V))
#        
#        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
#        output = self.W_o(self.combine_heads(attn_output))
#        return output

![Alt text](image.png)

In [ ]:
#class PositionWiseFeedForward(nn.Module):
#    def __init__(self, hidden_d, d_ff):
#        super(PositionWiseFeedForward, self).__init__()
#        self.fc1 = nn.Linear(hidden_d, d_ff)
#        self.fc2 = nn.Linear(d_ff, hidden_d)
#        self.relu = nn.ReLU()
#
#    def forward(self, x):
#        return self.fc2(self.relu(self.fc1(x)))

In [ ]:

#class VisualTransformerEncoderBlock(nn.Module):
#    '''
#    Not typical transformer block, the normalization and linear layers are there but the order is different.
#    '''
#    def __init__(self, hidden_d, n_heads, mlp_ratio=4):
#        super(VisualTransformerEncoderBlock, self).__init__()
#        self.hidden_d = hidden_d
#        self.n_heads = n_heads
#
#        self.norm1 = nn.LayerNorm(hidden_d)
#        self.mhsa = MultiHeadAttention(hidden_d, n_heads)
#        self.norm2 = nn.LayerNorm(hidden_d)
#        self.mlp = nn.Sequential(
#            nn.Linear(hidden_d, mlp_ratio * hidden_d),
#            nn.GELU(),
#            nn.Linear(mlp_ratio * hidden_d, hidden_d),
#        )
#
#    def forward(self, x):
#        out = x + self.mhsa(self.norm1(x), self.norm1(x), self.norm1(x)) 
#        out = out + self.mlp(self.norm2(out))
#        return out
#

In [ ]:
#class VisualTransformerEncoder(nn.Module):
#    def __init__(self, chw, n_patches=7, num_layers=2, hidden_d=8, n_heads=2):
#        # Super constructor
#        super(VisualTransformerEncoder, self).__init__()
#
#        # Attributes
#        self.chw = chw  # ( C , H , W )
#        self.n_patches = n_patches
#        self.num_layers = num_layers
#        self.n_heads = n_heads
#        self.hidden_d = hidden_d
#
#        # Input and patches sizes
#        assert (
#            chw[1] % n_patches == 0
#        ), "Input shape not entirely divisible by number of patches"
#        assert (
#            chw[2] % n_patches == 0
#        ), "Input shape not entirely divisible by number of patches"
#        self.patch_size = (chw[1] / n_patches, chw[2] / n_patches)
#
#        # 1) Linear mapper
#        self.input_d = int(chw[0] * self.patch_size[0] * self.patch_size[1])
#        self.linear_mapper = nn.Linear(self.input_d, self.hidden_d)
#
#        # 2) Learnable classification token
#        self.class_token = nn.Parameter(torch.rand(1, self.hidden_d))
#
#        # 3) Positional embedding
#        self.register_buffer(
#            "positional_embeddings",
#            get_positional_embeddings(n_patches**2, hidden_d),
#            persistent=False,
#        )
#
#        # 4) Transformer encoder blocks
#        self.blocks = nn.ModuleList(
#            [VisualTransformerEncoderBlock(hidden_d, n_heads) for _ in range(num_layers)]
#        )
#
#    def forward(self, images):
#        # Image to Patches
#        n, c, h, w = images.shape
#        patches = patchify(images, self.n_patches).to(self.positional_embeddings.device)
#
#        # Patch vector to Hidden Dimensions
#        tokens = self.linear_mapper(patches)
#
#        # Adding classification token to the tokens
#        #tokens = torch.cat((self.class_token.expand(n, 1, -1), tokens), dim=1)
#
#        # Adding positional embedding
#        out = tokens + self.positional_embeddings.repeat(n, 1, 1)
#
#        # Transformer Blocks
#        for block in self.blocks:
#            out = block(out)
#
#        return out, patches

![Alt text](image-1.png)

In [ ]:
#class VisualTransformerDecoderBlock(nn.Module):
#    def __init__(self, hidden_d, n_heads, d_ff, dropout):
#        super(VisualTransformerDecoderBlock, self).__init__()
#        self.self_attn = MultiHeadAttention(hidden_d, n_heads)
#        self.cross_attn = MultiHeadAttention(hidden_d, n_heads)
#        self.feed_forward = PositionWiseFeedForward(hidden_d, d_ff)
#        self.norm1 = nn.LayerNorm(hidden_d)
#        self.norm2 = nn.LayerNorm(hidden_d)
#        self.norm3 = nn.LayerNorm(hidden_d)
#        self.dropout = nn.Dropout(dropout)
#        
#    def forward(self, x, enc_output, src_mask, tgt_mask):
#        attn_output = self.self_attn(x, x, x, tgt_mask)
#        x = self.norm1(x + self.dropout(attn_output))
#        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
#        x = self.norm2(x + self.dropout(attn_output))
#        ff_output = self.feed_forward(x)
#        x = self.norm3(x + self.dropout(ff_output))
#        return x

In [ ]:
#class VisualTransformerDecoder(nn.Module):
#    def __init__(self, chw, d_ff, dropout, n_patches=7, num_layers=2, hidden_d=8, n_heads=2):
#        # Super constructor
#        super(VisualTransformerDecoder, self).__init__()
#
#        # Attributes
#        self.chw = chw  # ( C , H , W )
#        self.n_patches = n_patches
#        self.num_layers = num_layers
#        self.n_heads = n_heads
#        self.hidden_d = hidden_d
#        self.d_ff = d_ff
#        self.dropout = dropout
#        # Input and patches sizes
#        assert (
#            chw[1] % n_patches == 0
#        ), "Input shape not entirely divisible by number of patches"
#        assert (
#            chw[2] % n_patches == 0
#        ), "Input shape not entirely divisible by number of patches"
#        self.patch_size = (chw[1] / n_patches, chw[2] / n_patches)
#
#        # 1) Linear mapper
#        self.input_d = int(chw[0] * self.patch_size[0] * self.patch_size[1])
#        self.linear_mapper = nn.Linear(self.input_d, self.hidden_d)
#
#        # 2) Learnable classification token
#        self.class_token = nn.Parameter(torch.rand(1, self.hidden_d))
#
#        # 3) Positional embedding
#        self.register_buffer(
#            "positional_embeddings",
#            get_positional_embeddings(n_patches**2, hidden_d),
#            persistent=False,
#        )
#
#        # 4) Transformer encoder blocks
#        self.blocks = nn.ModuleList(
#            [VisualTransformerDecoderBlock(hidden_d, n_heads, d_ff, dropout) for _ in range(num_layers)]
#        )
#
#        self.linear_decoder = nn.Linear(self.hidden_d, self.input_d)
#
#    def forward(self, images,  enc_output, src_mask, tgt_mask):
#        # Image to Patches
#        n, c, h, w = images.shape
#        patches = patchify(images, self.n_patches).to(self.positional_embeddings.device)
#
#        # Patch vector to Hidden Dimensions
#        tokens = self.linear_mapper(patches)
#
#        # Adding classification token to the tokens
#        #tokens = torch.cat((self.class_token.expand(n, 1, -1), tokens), dim=1)
#
#        # Adding positional embedding
#        out = tokens + self.positional_embeddings.repeat(n, 1, 1)
#
#        # Transformer Blocks
#        for block in self.blocks:
#            out = block(out, enc_output, src_mask, tgt_mask)
#        
#        out = self.linear_decoder(out)
#
#        images = depatchify(out, self.n_patches, self.chw).to(self.positional_embeddings.device)
#
#        return images

In [ ]:
#class Transformer(nn.Module):
#    def __init__(self, hidden_d, n_heads, num_layers, d_ff, dropout, n_patches):
#        super(Transformer, self).__init__()
#
#        self.n_heads = n_heads
#        self.hidden_d = hidden_d
#        self.num_layers = num_layers
#        self.d_ff = d_ff
#        self.dropout = dropout
#        self.n_patches = n_patches
#
#        self.encoder = VisualTransformerEncoder((1, 28, 28), n_patches=self.n_patches, num_layers=self.num_layers, hidden_d=self.hidden_d, n_heads=self.n_heads)
#        self.decoder = VisualTransformerDecoder((1, 28, 28), d_ff=self.d_ff, dropout=self.dropout, n_patches=self.n_patches, num_layers=self.num_layers, hidden_d=self.hidden_d, n_heads=self.n_heads)
#
#        
#    def generate_mask(self, patches):
#        src = torch.ones((patches.size(0), patches.size(1)))
#        tgt = torch.ones((patches.size(0), patches.size(1)))
#
#        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
#        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
#        seq_length = tgt.size(1)
#        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
#        tgt_mask = tgt_mask & nopeak_mask
#
#        return src_mask, tgt_mask
#
#    def forward(self, images):
#        enc_output, patches = self.encoder(images)
#        src_mask, tgt_mask = self.generate_mask(patches)
#        src_mask = src_mask.to(device='mps')
#        tgt_mask = tgt_mask.to(device='mps')
#        dec_output = self.decoder(images, enc_output, src_mask, tgt_mask)
#        return dec_output

In [ ]:
#n_patches = 7
#num_layers = 4
#hidden_d = 64
#n_heads = 8
#d_ff = 512
#dropout = 0.1
#
#
#encoder = VisualTransformerEncoder((1, 28, 28), n_patches=n_patches, num_layers=num_layers, hidden_d=hidden_d, n_heads=n_heads).to(DEVICE)
###decoder_layers = nn.ModuleList([VisualTransformerDecoderBlock(hidden_d, n_heads, d_ff, dropout) for _ in range(num_layers)])
#decoder = VisualTransformerDecoder((1, 28, 28), d_ff=d_ff, dropout=dropout, n_patches=n_patches, num_layers=num_layers, hidden_d=hidden_d, n_heads=n_heads).to(DEVICE)
#
#
#images, classes = next(iter(train_loader))
#print(images.shape, classes.shape)
#patches = patchify(images, n_patches).unsqueeze(1)
#print(patches.shape)
#
#
#src_mask, tgt_mask = generate_mask(patches)
#src_mask = src_mask.to(DEVICE)
#tgt_mask = tgt_mask.to(DEVICE)
##src_embedded = positional_encoding(encoder_embedding(src))
##tgt_embedded = positional_encoding(decoder_embedding(tgt))
##enc_output = src_embedded
#enc_output = encoder(images)
#print('Encoded Latent Shape: {}'.format(enc_output.shape))
#dec_output = decoder(images, enc_output, src_mask, tgt_mask)
#print('Decoded Sequence Shape: {}'.format(dec_output.shape))

In [ ]:
n_patches = 7
num_layers = 4
hidden_d = 64
n_heads = 8
d_ff = 512
dropout = 0.1
learning_rate = 0.00001

transformer = Transformer(hidden_d, n_heads, num_layers, d_ff, dropout, n_patches).to(DEVICE)
criterion = nn.MSELoss().to(DEVICE)
optimizer = optim.Adam(transformer.parameters(), lr=learning_rate, eps=1e-9)
transformer.train()

for epoch in range(2):
    for batch in tqdm(train_dataloader):
        images, _ = batch
        images = images.to(DEVICE)
        
        optimizer.zero_grad()
        output = transformer(images)
        loss = criterion(images, output)
        loss.backward()
        optimizer.step()
    

    
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

In [ ]:
images = next(iter(val_dataloader))

In [ ]:
transformer.eval()
N=17
TensorToImage(transformer(images[0])[N].squeeze().detach().cpu())

In [ ]:
TensorToImage(images[0][N].squeeze().detach().cpu())

In [ ]:
#TensorToImage(depatchify(dec_output.detach().cpu(), n_patches, chw)[0].squeeze())
#TensorToImageGrid(depatchify(dec_output.detach().cpu(), n_patches, chw), rows=16, cols=8)
TensorToImageGrid(depatchify(patches.squeeze(1), n_patches, chw), rows=8, cols=16)


In [ ]:
TensorToImage(image_r)

In [ ]:
TensorToImage(images[0].squeeze())